In [1]:
import json
import pandas as pd
import pickle

In [2]:
# function to cheack whether an object (data) contains attribute (x) or not
def check_var(data, x) :
    try :
        data[x]
        return True
    except :
        return False

In [18]:
# function to turn json into dataframe
def json_to_df(file, hashtag) :
# we only keep text and hashtags
    df_list = []
    text_set = set()
    with open(file, 'r', encoding = 'utf8') as f :
        all_data = json.load(f)
     
        for data in all_data :
            hashtag_list = [] # to collect list of hashtags
            
            # case 1: check whether it is a retweet data or not
            if (check_var(data, 'retweeted_status')) :
                
                Text = data['retweeted_status']['full_text']
                for t in data['retweeted_status']['entities']['hashtags'] :
                    hashtag_list.append(t['text'])                

                # check whether it is retweet of quoted tweet or not
                if (check_var(data['retweeted_status'], 'quoted_status')) :
                    Text = Text + ' || ' + data['retweeted_status']['quoted_status']['full_text']
                    for t in data['retweeted_status']['entities']['hashtags'] :
                        hashtag_list.append(t['text'])               
 
            # case 2: check whether it is a quote tweet data or not
            elif (check_var(data, 'quoted_status')) :
                Text = data['full_text'] 
                Text = Text + ' || ' + data['quoted_status']['full_text']
                for t in data['entities']['hashtags'] :
                    hashtag_list.append(t['text'])
                for t in data['quoted_status']['entities']['hashtags'] :
                    hashtag_list.append(t['text'])

            # if it does not belong to case 1 and case 2, it is an ordinary tweet data
            else :
                Text = data['full_text']
                for t in data['entities']['hashtags'] :
                    hashtag_list.append(t['text'])
            
            hashtag_list = [h.lower() for h in hashtag_list]
            hashtag_list = [h for h in hashtag_list if h != hashtag] # remove our hashtag from hashtag_list    
            text_split = Text.lower().split() # split string into list
            text_split = [x for x in text_split if hashtag not in x] # remove all words containing our hashtag
            text_split = [x for x in text_split if not x.startswith('http')] # remove link
            text_split = [x for x in text_split if x != '\n'] # remove new line character
            Text = ' '.join(text_split) # join the list back to string
            
            for h in hashtag_list :
                Text = Text.replace('#' + h.lower(), h.lower()) # for other hashtags, we change them to words 
            
            if Text not in text_set : # check whether new data has already been kept in our list or not
                df_list.append((Text, hashtag_list))  
                text_set.add(Text)

    df = pd.DataFrame(df_list, columns = ['Text', 'Hashtag'])
    df['Classification'] = [hashtag] * df.shape[0]
    df = df[['Classification', 'Text', 'Hashtag']]
                    
    return df                

In [4]:
file = 'covid-out-23K.json'
covid_df = json_to_df(file, 'covid')

In [13]:
covid_df.head(10)

,Classification,Text,Hashtag
0,covid,"when i wonder how long we’ll be in lockdown, i...",[]
1,covid,so.... if a cure for has been found in just a ...,[cancer]
2,covid,"u.s deaths: 250,000-plus. (source: @johnshopki...",[]
3,covid,"why schools closing and bars, restaurants and ...","[nyc, schools]"
4,covid,my great-aunt died this evening of at the age ...,[covid_19]
5,covid,1 panelist suggests in scheduling quick virtua...,[remoteworking]
6,covid,allium vegetables for (sulfur) vegetable known...,[]
7,covid,the film “contagion” literally predicted every...,[covid19]
8,covid,anyone notice became the new fakenews msm topi...,"[fakenews, msm, election, voterfraud, wednesda..."
9,covid,@ksl5tv @pac12 @byufootball @utah_football @ks...,[]


In [6]:
file = 'christmas-out-23K.json'
christmas_df = json_to_df(file, 'christmas')

In [14]:
christmas_df.head(10)

,Classification,Text,Hashtag
0,christmas,is the perfect place for the holidays gift. su...,"[coldpresssoap, bodyscrub, soap, soapmaking, e..."
1,christmas,follow &amp; retweet for the chance to win a £...,"[competition, win]"
2,christmas,it's winitwednesday! follow &amp; rt @lauramar...,"[winitwednesday, win, astreetcatnamedbob, comp..."
3,christmas,there will be obstacles. there will be doubter...,"[cryptocurrency, binary, motivation, investor,..."
4,christmas,do you know how i plan to celebrate this year?...,"[durgapuja, diwali]"
5,christmas,meet the couples of bells pass! shep &amp; ivy...,"[amreadingromance, christmasromance, ku, roman..."
6,christmas,🌟win🌟 we’re now taking orders for both instore...,"[win, givingaway, prize, xmas]"
7,christmas,november blog post for harlequin romance inclu...,"[blog, harlequin, romance, reviews, romancerea..."
8,christmas,look at what we're taste testing today @choice...,"[mincepie, tasty]"
9,christmas,if you are in @inglesadvantage get the $5 food...,"[doyourpart, thanksgiving, christmasiscoming]"


In [19]:
file = 'spacex-out-35K.json'
spacex_df = json_to_df(file, 'spacex')

In [20]:
spacex_df.head(10)

,Classification,Text,Hashtag
0,spacex,"the return of crew-1's booster, b1061.1. after...","[crew2, spacexfleet, crew1, launchamerica]"
1,spacex,"weather for nov 21st, at 10:17pm est | 7:17pm ...","[starlink, falcon9]"
2,spacex,@apompliano falcon 9 landing ~ by harv 外人 @pun...,"[nft, cryptoart]"
3,spacex,moonship crew section.🚀🌒 moonship,[moonship]
4,spacex,looks like the left leg's crush core wasn't th...,[spacexfleet]
5,spacex,new development! starship sn9 nose cone forwar...,"[bocachica, starship]"
6,spacex,coming in febraury 2021 - just in time for the...,"[mars, space, science, marssociety, nasa]"
7,spacex,simply beautiful! || view from the @space_stat...,[]
8,spacex,i just really love this shit!,[]
9,spacex,: view from the @space_station of dragon‘s app...,[space]


In [10]:
with open("covid.txt", "wb") as handle : # write dataframe into text file
  pickle.dump(covid_df, handle)
with open("christmas.txt", "wb") as handle : # write dataframe into text file
  pickle.dump(christmas_df, handle)
with open("spacex.txt", "wb") as handle : # write dataframe into text file
  pickle.dump(spacex_df, handle)